In [1]:
import pandas as pd

In [2]:
scs = pd.read_csv('../data/SCS alerts Nov.csv')

In [3]:
scs.head()

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID
0,141254,C17: C1701STA254 254B12 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:13,3416
1,138183,C17: C1701STA012 012B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:03,3416
2,122518,C15: C1502STA034 034B12 occupancy sensor fault,01/11/2020 00:00,C15,P02,0.00:00:05,3416
3,444609,SCS M09: single module 1 emergency stop unit ...,01/11/2020 00:00,SCSM09,SCSM09,0.00:00:13,10408
4,139815,C17: C1701STA112 112B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:08,3416


In [4]:
scs['code'] = scs['Alert'].str.extract('(^[A-Z]{3}[0-9]{3}|[A-Z][0-9]{4}[A-Z]{3}[0-9]{3}|[A-Z]{3} [A-Z][0-9]{2})')
scs['code'] = scs['Alert'].str.extract('([A-Z][0-9]{4}[A-Z]{3}[0-9]{3})')

In [5]:
scs['code'].value_counts()

C1701STA112    3811
C1701STA254    2726
C2301STA175    2670
C2301RDC061    2481
C1404STA164    1961
               ... 
C0802STA074       1
C1601BCL182       1
C0904STA158       1
C1002ACH055       1
C1603ACH017       1
Name: code, Length: 1382, dtype: int64

### Failed Matches

In [6]:
scs['code'].isna().value_counts()

True     100635
False     73455
Name: code, dtype: int64

In [7]:
scs[scs['code'].isna()].head()

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID,code
3,444609,SCS M09: single module 1 emergency stop unit ...,01/11/2020 00:00,SCSM09,SCSM09,0.00:00:13,10408,NaN
6,439304,SCS228: load device: Schaefer Roll Gap B15 X:2...,01/11/2020 00:00,SCSM22,SCS228,0.00:00:07,5103,NaN
7,52405,C06: C06PTT031_ external unit fault,01/11/2020 00:00,C06,Z,0.00:00:16,3611,NaN
22,76581,C09: C09PTT102_ external unit fault,01/11/2020 00:02,C09,Z,0.00:00:05,3611,NaN
23,434103,SCS201: carousel: rotation area left B103 X:4...,01/11/2020 00:02,SCSM20,SCS201,0.00:04:07,5278,NaN


In [8]:
scs[scs['code'].isna()]['Desk'].value_counts()

Z         49033
SCSM12     1472
SCS184     1467
SCSM22     1452
SCS222     1385
          ...  
SCS095       54
SCS101       54
SCS053       52
SCS152       48
SCS106       41
Name: Desk, Length: 136, dtype: int64

In [9]:
bad_matches = scs[scs['code'].isna() & (scs['PLC'] != scs['Desk'])]['Alert']

In [10]:
scs.iloc[bad_matches[bad_matches.str.extract('([0-9]{2})')[0].astype('int') < 35].index].shape

(76139, 8)

In [13]:
scs.loc[scs['code'].isna(), 'code'] = scs.loc[scs['code'].isna(), 'PLC']

In [19]:
scs['Pick Station'] = scs['Alert'].str.extract('(PTT[0-9]{3})')[0]

In [21]:
scs['Pick Station'].value_counts()

PTT151    2879
PTT082    2474
PTT202    2401
PTT201    1673
PTT092    1450
PTT141    1425
PTT102    1337
PTT012    1284
PTT101    1259
PTT091    1248
PTT081    1166
PTT172    1134
PTT031    1116
PTT072    1107
PTT182    1076
PTT042    1076
PTT181    1050
PTT131    1014
PTT192    1007
PTT191    1007
PTT071     982
PTT171     935
PTT142     903
PTT032     887
PTT132     847
PTT022     839
PTT041     835
PTT122     753
PTT011     747
PTT152     743
PTT112     731
PTT051     641
PTT111     601
PTT021     534
PTT052     455
PTT121     436
Name: Pick Station, dtype: int64

In [12]:
scs.to_csv('../data/SCS alerts Nov_with_asset_code.csv', index=False)